In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

paths = list()
base_path = '/kaggle/input/uplift-shift-23/x5-uplift-valid/'
for dirname, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        paths.append(path)
        print(path)

/kaggle/working/__notebook_source__.ipynb


In [2]:
# это OHE, которая оставляется только лейблы с определенным уровнем представленности
def my_dummies(initial_column, threshold = None, sparse = False):
    column = initial_column.copy()
    if threshold is not None:
        count = pd.value_counts(column)
        filtred_column = count[count > threshold] # возможно, есть проблема с кодировкой np.NaN
        mask = column.isin(filtred_column.index)
        column[~mask] = 'DELETE_ME'
        print(f'{column.name: >10}: {mask.sum(): >5} of {len(column): >5} ({mask.sum()/len(column):0.7f}) '\
              f'making {len(filtred_column)}/{len(initial_column.unique())-1} columns')
    else:
        print(f'{column.name: >10}: is fully encoded in {len(column.unique())} columns')
        
    return pd.get_dummies(column, prefix=column.name, prefix_sep='_', sparse=sparse).drop(f'{column.name}_DELETE_ME', axis=1, errors='ignore')

In [3]:
# пригодится для приведения данных к численном виду
def safe_timestamp(timestamp) -> int:
    try:
        return timestamp.timestamp()
    except ValueError:
        return np.nan

In [4]:
%%time
# читаем данные о клиентах
clients = pd.read_csv(f'{base_path}data/clients2.csv').drop(columns=['client_id.1'])

# приводим даты к численному виду
clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'], format='%Y-%m-%d %H:%M:%S').apply(safe_timestamp)
clients['first_redeem_date'] = pd.to_datetime(clients['first_redeem_date'], format='%Y-%m-%d %H:%M:%S').apply(safe_timestamp)

# заполняем пропущенные даты нулями
clients['first_redeem_date'].fillna(0)

# OHE пола клиентов
clients = pd.concat([clients, pd.get_dummies(clients['gender'], prefix='gender', prefix_sep='_')], axis=1).drop(['gender', 'gender_U'], axis=1)

# устаналиваем идентификатор клиента в качестве индекса
clients.set_index('client_id', inplace=True)

CPU times: user 2.92 s, sys: 183 ms, total: 3.1 s
Wall time: 3.28 s


In [5]:
%%time
# читаем данные о товарах, делая OHE с учетом предствленности лейблов
INF = 10 ** 10 # пороговое число, при указании которого у нас ничто не закодируется
products = pd.read_csv(f'{base_path}data/products.csv')
for column_name, num in (('level_1', None),('level_2', 200),('level_3', 200),('level_4', 200),('segment_id', 300),('vendor_id', INF),('brand_id', 500)):
    products = pd.concat([products, my_dummies(products[column_name], num)], axis=1).drop(column_name, axis=1)

   level_1: is fully encoded in 4 columns
   level_2: 42069 of 43038 (0.9774850) making 26/42 columns
   level_3: 34341 of 43038 (0.7979228) making 48/201 columns
   level_4: 23889 of 43038 (0.5550676) making 47/790 columns
segment_id: 32543 of 43038 (0.7561457) making 40/116 columns
 vendor_id:     0 of 43038 (0.0000000) making 0/3193 columns
  brand_id:  7415 of 43038 (0.1722896) making 2/4296 columns
CPU times: user 342 ms, sys: 43.9 ms, total: 386 ms
Wall time: 458 ms


In [6]:
%%time

version = 'v4'
chunk = 10 ** 6

for iteration_num in range(16+7):  # сдвиг, с помощью которого мы по частями обрабатываем данные
    data_type = 'train_purch' if iteration_num < 16 else 'test_purch' # 15`998`952 + 6`883`738
    print(f'{iteration_num+1: >2} iteration, {data_type}.csv ...')
    
    # читаем чанк данных одного из файлов
    purch_data = pd.read_csv(f'{base_path}{data_type}/{data_type}.csv', skiprows=(chunk)*(iteration_num%16)+1, nrows=chunk, 
                             names=['client_id', 'transaction_id', 'transaction_datetime', 'regular_points_received', 
                                    'express_points_received', 'regular_points_spent', 'express_points_spent', 
                                    'purchase_sum', 'store_id', 'product_id', 'product_quantity', 'trn_sum_from_iss',
                                    'trn_sum_from_red'])
    
    # меняем purchase_sum на сумму с учетом product_quantity
    purch_data['purchase_true_sum'] = purch_data.apply(lambda df: df['purchase_sum'] * df['product_quantity'], axis=1)

    # учитываем что-то типа популярности магазина (ясно, что обрабатывая данные чанками получаем лишь часть статистики)
    store_id_count = pd.value_counts(purch_data['store_id']).to_dict()
    purch_data['store_count'] = purch_data['store_id'].apply(lambda x: store_id_count[x]) 
    purch_data.drop('store_id', axis=1, inplace=True)

    # конвертация времени в чиселку
    purch_data['transaction_datetime'] = pd.to_datetime(purch_data['transaction_datetime'], format='%Y-%m-%d %H:%M:%S').apply(safe_timestamp)

    # кажется, тут можно просто заполнять нулями
    purch_data.fillna(0, inplace=True)

    # соединяем с данными о продуктах
    purch_data = purch_data.merge(products, how='left', on='product_id').drop('product_id', axis=1)
    
    # заготавливаем словарик с функциями агрегации
    agg_trans = {name: 'mean' for name in purch_data.columns if name != 'transaction_id'}
    agg_trans['store_count'] = agg_trans['transaction_datetime'] = agg_trans['client_id'] = 'first'
    agg_trans['purchase_sum'] = agg_trans['purchase_true_sum'] = agg_trans['product_quantity'] = ['sum', 'mean']

    # применяем подготовленные аггрегационные функции
    grouped_trans = purch_data.groupby('transaction_id')
    transactions = pd.concat([grouped_trans.agg(agg_trans), grouped_trans['product_quantity'].size()], axis=1)

    # переходим от tuple к str в качестве имен колонок
    transactions.rename(columns={column: '_'.join(column) for column in transactions.columns if isinstance(column, tuple)}, inplace=True)

    # усредняем данные по каждому клинету (кое-что суммируем)
    grouped_clients = transactions.groupby('client_id_first')
    clients_purches = pd.concat([grouped_clients.mean(), 
                                 grouped_clients['product_quantity'].size(), # кол-во походов в магазин
                                 grouped_clients['product_quantity_sum'].sum(), # общее кол-во купленных товаров
                                 grouped_clients['purchase_true_sum_sum'].sum(), # общее кол-во потраченных денег
                                 grouped_clients['purchase_sum_sum'].sum(), # общее кол-во потраченных денег (не учитывает кол-во товара)
                                 grouped_clients['transaction_datetime_first'].std(), # "кучность" походов в магазин
                                 grouped_clients['transaction_datetime_first'].max() - 
                                 grouped_clients['transaction_datetime_first'].min() # период, в течение которого клиент ходил в магазин
                                 ], axis=1)
    
    # переименуем индекс-колонку
    clients_purches.index.rename('client_id', inplace=True)

    # добавляем информацию о самих клиентах 
    result = clients.merge(clients_purches, how='right', on='client_id')
    
    # пишем один чанк данных
    result.to_csv(f'/kaggle/working/purches_{version}.csv', mode='w' if iteration_num == 0 else 'a', header=False)

 1 iteration, train_purch.csv ...
 2 iteration, train_purch.csv ...
 3 iteration, train_purch.csv ...
 4 iteration, train_purch.csv ...
 5 iteration, train_purch.csv ...
 6 iteration, train_purch.csv ...
10 iteration, train_purch.csv ...
11 iteration, train_purch.csv ...
12 iteration, train_purch.csv ...
13 iteration, train_purch.csv ...
14 iteration, train_purch.csv ...
15 iteration, train_purch.csv ...
16 iteration, train_purch.csv ...
17 iteration, test_purch.csv ...
18 iteration, test_purch.csv ...
19 iteration, test_purch.csv ...
20 iteration, test_purch.csv ...
21 iteration, test_purch.csv ...
22 iteration, test_purch.csv ...
23 iteration, test_purch.csv ...
CPU times: user 14min 30s, sys: 35.6 s, total: 15min 6s
Wall time: 15min 35s


In [7]:
# смотрим, что мы там сохранили
purches = pd.read_csv(f'/kaggle/working/purches_{version}.csv', header=None)

In [8]:
# ну вот на стыках чанков у нас получилось разделение списков покупок покупателей (21 таких)
clients_ids = purches.iloc[:, 0].tolist()
print(len(clients_ids), len(set(clients_ids)))

# и это как раз согласуется с количеством уникальных client_id
print(pd.read_csv(f'{base_path}data/train.csv').shape[0] + pd.read_csv(f'{base_path}data/test.csv').shape[0])

200060 200039
200039


In [18]:
# просто выкинем любую из дублирующих строк (21 испорченный сэмпл - не так страшно)
purches.drop_duplicates(subset=[0], inplace=True)

# надо еще в паре столбцов заполнить пропуски
purches.fillna(0, inplace=True)

In [10]:
# запишем итоговый предобработанный датасет
purches.to_csv(f'/kaggle/working/purches_{version}.csv', header=False)